In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score, GridSearchCV, train_test_split

class DayForwardSplit:
    def __init__(self, test_size, train_size, n_splits=None):
        self.n_splits = n_splits
        self.test_size = test_size
        self.train_size = train_size
    
    def get_n_splits(self, X, y, groups):
        if self.n_splits == None:
            self.n_splits = len(X) - self.train_size - self.test_size + 1
        return self.n_splits
        
    def split(self, X, y=None, groups=None):
        rightmost = len(X)
        if self.n_splits == None:
            self.n_splits = rightmost - self.train_size - self.test_size + 1
        indices = np.arange(rightmost)
        for i in range(self.n_splits):
            right = rightmost - self.n_splits + i + 1
            mid = right - self.test_size
            left = mid - self.train_size
            yield indices[left:mid],indices[mid:right]

## data

In [2]:
prices=pd.read_csv("./data/prices.csv", index_col=0,parse_dates=True)
prices.columns = prices.columns.str.strip()
prices['AMZN'] = prices.AMZN.str.strip().str.replace(",","").astype(float)
returns = prices.pct_change() * 100
factors=pd.read_csv("./data/factors.csv", index_col=0,parse_dates=True)
df=factors.merge(returns,left_index=True, right_index=True).dropna()

## use cross validation to find the best lookback period for simple OLS

#### train test split

In [3]:
col_X = factors.columns.drop("RF")
col_y = 'AAPL'
X_train, X_test, y_train, y_test = train_test_split(df[col_X], df[col_y], test_size=0.05, shuffle=False)

In [4]:
len(X_test)

38

#### simple OLS cross validation
#### because TimeSeriesSplit is set to have 5 splits, we do 5 experiments with train dateset in the way specified by TimeSeriesSplit
#### And we have scores for these 5 experiments
#### The evaluation metric here is R2, between 0 and 1. If R2 is 0.228, it means we get a bad OLS model if we use a part of train set to train our OLS model (get factor loadings or slopes), and use these parameters to predict  estimated y = b1 * x1 + b2 * x2 + ..., and compare them with actual values
#### Take average to get the final cv score

In [5]:
model=LinearRegression()
cross_val_score(model, X_train, y_train, cv=TimeSeriesSplit(n_splits=5), scoring="r2")

array([0.22804368, 0.80392258, 0.64397992, 0.64737597, 0.69987955])

In [6]:
# custom estimator: a simple linear regression, but force to restrict the size of training set
class OLS_with_lookback(LinearRegression):
    def __init__(self, lookback_period=10):
        self.lookback_period = lookback_period
        self.model = LinearRegression()
    def fit(self, X, y):
        self.model.fit(X.iloc[-self.lookback_period:],y.iloc[-self.lookback_period:])
        return self
    def predict(self,X):
        return self.model.predict(X)

In [7]:
cross_val_score(OLS_with_lookback(lookback_period=30), X_train, y_train, cv=TimeSeriesSplit(n_splits=5), scoring="r2")

array([0.45668941, 0.8487599 , 0.55128194, 0.6928377 , 0.62683252])

#### Now we can try different values of lookback period. For each lookback period, we can do 5 experiments. 
#### Here in total 3 possible choices * 5 experiments = 15 fits

In [8]:
model=OLS_with_lookback()
param_grid={
    "lookback_period":[10,20,30]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=TimeSeriesSplit(n_splits=5), verbose=4)
GS.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...........lookback_period=10; r2: (test=-0.171) total time=   0.0s
[CV 2/5] END ............lookback_period=10; r2: (test=0.509) total time=   0.0s
[CV 3/5] END ...........lookback_period=10; r2: (test=-1.040) total time=   0.0s
[CV 4/5] END ............lookback_period=10; r2: (test=0.086) total time=   0.0s
[CV 5/5] END ............lookback_period=10; r2: (test=0.062) total time=   0.0s
[CV 1/5] END ............lookback_period=20; r2: (test=0.486) total time=   0.0s
[CV 2/5] END ............lookback_period=20; r2: (test=0.808) total time=   0.0s
[CV 3/5] END ............lookback_period=20; r2: (test=0.322) total time=   0.0s
[CV 4/5] END ............lookback_period=20; r2: (test=0.485) total time=   0.0s
[CV 5/5] END ............lookback_period=20; r2: (test=0.691) total time=   0.0s
[CV 1/5] END ............lookback_period=30; r2: (test=0.457) total time=   0.0s
[CV 2/5] END ............lookback_period=30; r2: 

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=OLS_with_lookback(),
             param_grid={'lookback_period': [10, 20, 30]}, refit='r2',
             scoring=['r2'], verbose=4)

In [9]:
GS.best_estimator_

OLS_with_lookback(lookback_period=30)

In [10]:
GS.best_score_

0.6352802916282163

#### Among lookback period (10,20,30), 30 is the best one. We should definitely try a larger number

#### Try 10 values

In [11]:
len(X_train)

718

In [12]:
step_size=0.1
np.arange(step_size,1+step_size,step_size)

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [13]:
np.arange(step_size,1+step_size,step_size) * len(X_train)

array([ 71.8, 143.6, 215.4, 287.2, 359. , 430.8, 502.6, 574.4, 646.2,
       718. ])

In [14]:
[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * len(X_train)]

[71, 143, 215, 287, 359, 430, 502, 574, 646, 718]

In [15]:
step_size=0.1
model=OLS_with_lookback()
param_grid={
    "lookback_period":[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * len(X_train)]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=TimeSeriesSplit(n_splits=5), verbose=0)
GS.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=OLS_with_lookback(),
             param_grid={'lookback_period': [71, 143, 215, 287, 359, 430, 502,
                                             574, 646, 718]},
             refit='r2', scoring=['r2'])

In [16]:
GS.best_estimator_

OLS_with_lookback(lookback_period=143)

In [17]:
GS.best_score_

0.6290230299059646

#### Try 100 values

In [18]:
step_size=0.01
model=OLS_with_lookback()
param_grid={
    "lookback_period":[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * len(X_train)]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=TimeSeriesSplit(n_splits=5), verbose=0)
GS.fit(X_train, y_train)
print(GS.best_estimator_, '\tR2:', GS.best_score_)

OLS_with_lookback(lookback_period=57) 	R2: 0.6454369951662746


## Is that correct? No! Let's take a look at what TimeSeriesSplit is doing

#### In the first experiment, we use 5 numbers to trian a model, and use the model to predict 5 new numbers. Riduculous...
#### Recall we had R2 equals about 0.2, while the remaining R2 scores are much bigger

In [19]:
arr=list(range(20))
time_series_split = TimeSeriesSplit(n_splits=3)
for i in time_series_split.split(arr):
    print(i)

(array([0, 1, 2, 3, 4]), array([5, 6, 7, 8, 9]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([10, 11, 12, 13, 14]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([15, 16, 17, 18, 19]))


#### Instead, set test_size and max_train_size
#### test_size: 63 days. We assume 21 trading days every months, 63 days = 3 months
#### max_train_size: 375 days. About 1.5 years

In [20]:
arr=list(range(len(X_train)))
time_series_split = TimeSeriesSplit(n_splits=5,test_size=63,max_train_size=375)
for i in list(time_series_split.split(arr))[:2]:
    print(i)

(array([ 28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
       197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 2

#### After configuring TimeSeriesSplit correctly, we have reasonable R2 scores in the first experiment.
#### However, we get R2 = 0.42. Why? The lookback period is 30 while the test size is 63. So we use 1 month data to predict 2 month.

In [21]:
cross_val_score(OLS_with_lookback(lookback_period=30), X_train, y_train, 
                cv=TimeSeriesSplit(n_splits=5,test_size=63,max_train_size=375), 
                scoring="r2")

array([0.71329476, 0.42117279, 0.77725839, 0.63432326, 0.68045854])

#### Switch to lookback period = 380
#### Now looks okay

In [22]:
cross_val_score(OLS_with_lookback(lookback_period=380), X_train, y_train, 
                cv=TimeSeriesSplit(n_splits=5,test_size=63,max_train_size=375), 
                scoring="r2")

array([0.72585668, 0.61666288, 0.69246936, 0.77646367, 0.71607496])

In [23]:
cross_val_score(OLS_with_lookback(lookback_period=400), X_train, y_train, 
                cv=TimeSeriesSplit(n_splits=5,test_size=63,max_train_size=375), 
                scoring="r2")

array([0.72585668, 0.61666288, 0.69246936, 0.77646367, 0.71607496])

#### try 100 values, n_splits = 3
#### result is that, we use 315 days to do simple linear regression, and that has the optimal fitting with R2 = 0.7445

In [24]:
max_train_size=375 # 1.5 years
test_size=42 # 2 months
n_splits=3
cv_split = TimeSeriesSplit(n_splits=n_splits,max_train_size=max_train_size,test_size=test_size)

step_size=0.01
model=OLS_with_lookback()
param_grid={
    "lookback_period":[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * max_train_size]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=cv_split, verbose=0)
GS.fit(X_train, y_train)
print(GS.best_estimator_, '\tR2:', GS.best_score_)

OLS_with_lookback(lookback_period=315) 	R2: 0.7445318252617107


#### try 100 values, n_splits = 5; n_splits = 10
#### The best lookback period becomes 135 and 277. Quite unstable!

In [25]:
max_train_size=375 # 1.5 years
test_size=42 # 2 months
n_splits=5
cv_split = TimeSeriesSplit(n_splits=n_splits,max_train_size=max_train_size,test_size=test_size)

step_size=0.01
model=OLS_with_lookback()
param_grid={
    "lookback_period":[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * max_train_size]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=cv_split, verbose=0)
GS.fit(X_train, y_train)
print(GS.best_estimator_, '\tR2:', GS.best_score_)

OLS_with_lookback(lookback_period=135) 	R2: 0.7326070465915823


In [26]:
max_train_size=375 # 1.5 years
test_size=42 # 2 months
n_splits=10
cv_split = TimeSeriesSplit(n_splits=n_splits,max_train_size=max_train_size,test_size=test_size)

step_size=0.01
model=OLS_with_lookback()
param_grid={
    "lookback_period":[int(elem) for elem in np.arange(step_size,1+step_size,step_size) * max_train_size]
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=cv_split, verbose=0)
GS.fit(X_train, y_train)
print(GS.best_estimator_, '\tR2:', GS.best_score_)

OLS_with_lookback(lookback_period=277) 	R2: 0.7140791952700191


## Now what to do? Use DayForwardSplit, exhausting all of the experiments to give a robust value

In [27]:
day_forward_split = DayForwardSplit(test_size=2, train_size=7, n_splits=3)
for i in day_forward_split.split(arr):
    print(i)

(array([707, 708, 709, 710, 711, 712, 713]), array([714, 715]))
(array([708, 709, 710, 711, 712, 713, 714]), array([715, 716]))
(array([709, 710, 711, 712, 713, 714, 715]), array([716, 717]))


In [28]:
# Set n_splits = 3
day_forward_split = DayForwardSplit(test_size=42, train_size=375, n_splits=3)
for i in day_forward_split.split(arr):
    print(i)

(array([299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311,
       312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324,
       325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337,
       338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350,
       351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
       364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376,
       377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389,
       390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402,
       403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415,
       416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428,
       429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441,
       442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454,
       455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467,
       468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 4

In [29]:
# Not set n_splits, it exhausts all of the experiments
day_forward_split = DayForwardSplit(test_size=42, train_size=375)
for i in list(day_forward_split.split(arr))[0]:
    print(i)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [30]:
# We try lookback_periods starting from a reasonable value, such as starting from twice the test size
train_size=375
lookback_periods = np.arange(step_size,1+step_size,step_size) * train_size
lookback_periods[lookback_periods > (42*2)]

array([ 86.25,  90.  ,  93.75,  97.5 , 101.25, 105.  , 108.75, 112.5 ,
       116.25, 120.  , 123.75, 127.5 , 131.25, 135.  , 138.75, 142.5 ,
       146.25, 150.  , 153.75, 157.5 , 161.25, 165.  , 168.75, 172.5 ,
       176.25, 180.  , 183.75, 187.5 , 191.25, 195.  , 198.75, 202.5 ,
       206.25, 210.  , 213.75, 217.5 , 221.25, 225.  , 228.75, 232.5 ,
       236.25, 240.  , 243.75, 247.5 , 251.25, 255.  , 258.75, 262.5 ,
       266.25, 270.  , 273.75, 277.5 , 281.25, 285.  , 288.75, 292.5 ,
       296.25, 300.  , 303.75, 307.5 , 311.25, 315.  , 318.75, 322.5 ,
       326.25, 330.  , 333.75, 337.5 , 341.25, 345.  , 348.75, 352.5 ,
       356.25, 360.  , 363.75, 367.5 , 371.25, 375.  ])

#### Now we step through every single day, and average out. The optimal lookback period is 150 days.

In [31]:
train_size=375 # 1.5 years
test_size=len(X_test) # 38 days, about 2 months
cv_split = DayForwardSplit(test_size=test_size, train_size=train_size)

step_size=0.05 # try fewer than 20 values
lookback_periods = np.arange(step_size,1+step_size,step_size) * train_size
lookback_periods = lookback_periods[lookback_periods>(test_size*2)]
lookback_periods = [int(elem) for elem in lookback_periods]

model=OLS_with_lookback()
param_grid={
    "lookback_period":lookback_periods
}
GS = GridSearchCV(model, param_grid=param_grid, scoring=['r2'],refit='r2', cv=cv_split, verbose=0)
GS.fit(X_train, y_train)
print(GS.best_estimator_, '\tR2:', GS.best_score_)

OLS_with_lookback(lookback_period=150) 	R2: 0.7135202593640676


In [32]:
cross_val_score(OLS_with_lookback(lookback_period=GS.best_params_['lookback_period']), X_train, y_train, 
                cv=cv_split, 
                scoring="r2")

array([0.50769419, 0.51152577, 0.52596444, 0.5271933 , 0.52260045,
       0.51751326, 0.50095947, 0.49876324, 0.44048238, 0.55561064,
       0.54861397, 0.59350074, 0.60399981, 0.61514315, 0.60924744,
       0.60786355, 0.60349948, 0.59992666, 0.60264873, 0.61132162,
       0.59237095, 0.58826297, 0.60636865, 0.70319291, 0.70837702,
       0.71107169, 0.71157807, 0.71145209, 0.71375452, 0.72253729,
       0.73279618, 0.7307543 , 0.73834509, 0.73599658, 0.74109795,
       0.74052362, 0.73704647, 0.73544085, 0.74924516, 0.74340398,
       0.75099508, 0.75069904, 0.75426271, 0.75824429, 0.76929737,
       0.75728946, 0.81412083, 0.79821527, 0.80234156, 0.78214512,
       0.75899323, 0.75196585, 0.77431355, 0.78120933, 0.78507326,
       0.78623116, 0.78805066, 0.78325552, 0.81073522, 0.80294377,
       0.7932593 , 0.79423136, 0.79445674, 0.79183713, 0.79282332,
       0.75664626, 0.75784858, 0.74669961, 0.73968475, 0.73458606,
       0.73966161, 0.74466216, 0.74582568, 0.73169824, 0.72858

#### Finanly use lookback period = 150 to train OLS model. It gives 1.218
#### Recall that the true market beta in the following 2 months is: 1.147, and a naive OLS tells us 1.330

In [33]:
res=[]
lookback_period=150
reg = LinearRegression()
col_factors=factors.columns.drop("RF")
col_X = col_factors
col_y = 'AAPL'
for i in range(len(df)-lookback_period+1):
    sub=df.iloc[i:i+lookback_period] # until i+window=length, i can be at most len(df)-window
    X=sub[col_X]
    y=sub[col_y]
    res.append(reg.fit(X,y).coef_)
    
factor_loadings = pd.DataFrame(res,index=df.index[-(len(df)-lookback_period)-1:],columns=col_factors)
print(factor_loadings)

              Mkt-RF       SMB       HML       RMW       CMA
2019-08-07  1.282995 -0.294146 -0.220476  1.044175 -1.644604
2019-08-08  1.149216 -0.168831 -0.164406  0.867413 -1.488239
2019-08-09  1.180150 -0.172586 -0.169873  0.876044 -1.470057
2019-08-12  1.149010 -0.105213 -0.207170  0.837370 -1.477146
2019-08-13  1.196210 -0.134327 -0.228185  0.880178 -1.406082
...              ...       ...       ...       ...       ...
2021-12-27  1.220993 -0.149857 -1.022274  0.414247  1.774409
2021-12-28  1.218153 -0.155688 -1.038234  0.411429  1.796065
2021-12-29  1.218192 -0.162449 -1.037662  0.407325  1.798604
2021-12-30  1.216289 -0.150146 -1.032113  0.405697  1.791791
2021-12-31  1.217871 -0.154059 -1.029039  0.403398  1.783543

[607 rows x 5 columns]


In [34]:
df_factor_lodings = pd.concat([
        factor_loadings['Mkt-RF'].rolling(window=42).mean(),
        factor_loadings['Mkt-RF'].ewm(span=10).mean(),
        factor_loadings['Mkt-RF']
],axis=1)
df_factor_lodings.columns=['sma','ewa','raw']
print(df_factor_lodings)

                 sma       ewa       raw
2019-08-07       NaN  1.282995  1.282995
2019-08-08       NaN  1.209416  1.149216
2019-08-09       NaN  1.197652  1.180150
2019-08-12       NaN  1.181626  1.149010
2019-08-13       NaN  1.185813  1.196210
...              ...       ...       ...
2021-12-27  1.282332  1.217096  1.220993
2021-12-28  1.280011  1.217289  1.218153
2021-12-29  1.276528  1.217453  1.218192
2021-12-30  1.273008  1.217241  1.216289
2021-12-31  1.269407  1.217356  1.217871

[607 rows x 3 columns]


#### sma52 is 1.2694, ewa10 is 1.2174
#### Well we just went through 1 hyperpamater; we can also do the same thing to the window size of sma, or the span size of ewa
#### We can try different combinations of (lookback period, sma window)
#### If lookback period is 150, sma window is 2

In [35]:
LinearRegression().fit(X_train.iloc[-151:-1],y_train.iloc[-151:-1]).coef_

array([ 1.37745128, -0.37758642, -0.80375525,  0.19904453,  1.52987811])

In [36]:
LinearRegression().fit(X_train.iloc[-150:],y_train.iloc[-150:]).coef_

array([ 1.37665808, -0.37699977, -0.79393917,  0.20906179,  1.4754982 ])

In [37]:
(1.37745128 + 1.37665808) / 2

1.37705468